In [98]:
import pandas as pd
# !pip install pyarrow

MESES = ['01','02','03','04','05','06','07','08','09','10','11','12']
ANOS = ['2022','2023']
COL_INTERESSE = ['Matricula','Nome','Orgão','Setor','Cargo','Provento','Date']


In [99]:
###Definindo Função para criar link dos aquivos desejados###
def url_mes_ano (mes, ano):
    mes=str(mes)
    ano=str(ano)
    api_url_prefix='https://api.portaltransparencia.sead.caucaia.ce.gov.br/funcionario/excel/1/'
    api_url_final=api_url_prefix + mes + '/' + ano
    return api_url_final


In [100]:
#### criando funcao pra coletar todos os dados ####
def download_all_data(meses:list, anos:list):
    df_geral = pd.DataFrame()
    for ano in anos:
        for mes in meses:
            try:
                df = pd.read_excel(url_mes_ano(mes,ano), header=1) # ler o arquivo diretamente do link gerado
                df['Date'] = pd.to_datetime(f'{ano}-{mes}-01') #cria coluna do mes e ano no formato ce data
                df_geral = pd.concat([df_geral,df])
                
            except:
                pass
    df = df_geral.reset_index()# retorna o DF com os indices corrigidos

    return df[COL_INTERESSE]

In [101]:
def converter_texto_em_float(texto:str):
    try:
        texto = texto.replace('R$','').strip()
        texto = texto.replace(',','.')
        return float(texto)
    except:
        return texto
    


In [102]:
def limpar_dados(df:pd.DataFrame):
    df = df.dropna(subset=['Nome']) #limpa as linhas que nao possuem nomes
    df['Provento'] = df['Provento'].map(converter_texto_em_float)#converter valores que possuem prefixo R$ em float
    #removendo no nomes cujos proventos sao igual a zero
    index_proventos_nulos = df[df['Provento']== 0].index.to_list() #coletando a lista de indices cujos os proventos sao zero
    for indice in index_proventos_nulos:
        df = df.drop(index=indice) #remove cada indice da lista
    df = df.reset_index() #reseta index para a ordem correta
    df = df[COL_INTERESSE]

    return df


In [103]:
#### carregando arquivos ###
file_name = 'dados.feather'

try:
    df = pd.read_feather(file_name)
except:
    print('Dados não encontrados. Realizando o download...')
    ####fazendo o download de tudo e salvando #####
    df = download_all_data(meses=MESES,anos=ANOS)
    df = limpar_dados(df)
    df.to_feather(file_name)
# df = limpar_dados(df=df)


In [104]:
#maximo valor pago a prefeitura
provento_max_global = df.describe().to_dict()['Provento']['max']
provento_mean_global = df.describe().to_dict()['Provento']['mean']

print(f'Provento maximo: R${provento_max_global}')
print(f'Provento medio: R${provento_mean_global:.2f}')
#mes com mais proventos
df_aux = df.groupby([pd.Grouper(key='Date',axis=0, freq='M')]).sum().sort_values('Provento', ascending=False).head(1)
provento_max_pago_mes_numero = df_aux['Provento'].keys().month.to_list()[0]
provento_max_pago_ano_numero = df_aux['Provento'].keys().year.to_list()[0]
provento_max_pago_mes_valor = df_aux['Provento'].to_list()[0]

print(f'O mairo valor pago foi no mes {provento_max_pago_mes_numero}/{provento_max_pago_ano_numero}, no valor de: R${provento_max_pago_mes_valor}')



Provento maximo: R$89166.66
Provento medio: R$4767.51
O mairo valor pago foi no mes 10/2022, no valor de: R$108521808.42


In [137]:
df_aux = df[['Setor', 'Provento']].groupby('Setor').sum().sort_values('Provento', ascending=False)
setor_maior_gasto = df_aux.head(1).index.to_list()[0]

# df_gastos_setor = df_aux.reset_index()
df_gastos_setor = df.groupby(['Setor','Date']).sum().reset_index()
del df_aux
df_gastos_setor['Setor'] = df_gastos_setor['Setor'].map(lambda x : x.strip()[:20])

df_gastos_setor.head(10)

,Setor,Date,Matricula,Nome,Orgão,Cargo,Provento
0,002-GABINETE DO PREF,2022-02-01,3053050,ANTONIO ZEUDIR QUEIROZ SOUTOFRANCISCO JOSE GOU...,GABINETE DO PREFEITOGABINETE DO PREFEITOGABINE...,AGENTE DE SUPORTE GERENCIALECONOMISTAAGENTE DE...,264900.86
1,002-GABINETE DO PREF,2022-03-01,81544,ANTONIA DA PENHA SIMAO DOS SANTOSEDINALVA QUEI...,GABINETE DO PREFEITOGABINETE DO PREFEITOGABINE...,AUXILIAR OPERACIONALAGENTE DE SUPORTE GERENCIA...,8860.92
2,002-GABINETE DO PREF,2022-05-01,3634386,ALINE ELLEN UMBELINO DE SOUSAFRANCISCO ROGERIO...,GABINETE DO PREFEITOGABINETE DO PREFEITOGABINE...,ASSESSOR ESPECIAL IIAGENTE DE SUPORTE GERENCIA...,298272.30
3,002-GABINETE DO PREF,2022-06-01,3478531,FRANCISCO ROGERIO MARTINS V DO VALECARLOS CRIS...,GABINETE DO PREFEITOGABINETE DO PREFEITOGABINE...,AGENTE DE SUPORTE GERENCIALASSESSOR ESPECIAL I...,294083.49
4,002-GABINETE DO PREF,2022-07-01,3562311,GISELLE RENATA ALENCAR DE AQUINOJOANA MARIANA ...,GABINETE DO PREFEITOGABINETE DO PREFEITOGABINE...,GERENTE DE UNIDADEASSESSOR ESPECIAL IICHEFE DE...,298697.50
5,002-GABINETE DO PREF,2022-08-01,3730188,JANETE CLARICE AMARAL DA COSTA ARNDTFRANCISCO ...,GABINETE DO PREFEITOGABINETE DO PREFEITOGABINE...,ASSESSOR ESPECIAL IIAGENTE DE SUPORTE GERENCIA...,311679.90
6,002-GABINETE DO PREF,2022-09-01,3694960,JANETE CLARICE AMARAL DA COSTA ARNDTJOSE PINTO...,GABINETE DO PREFEITOGABINETE DO PREFEITOGABINE...,ASSESSOR ESPECIAL IIGERENTE DE NUCLEOAGENTE DE...,316753.96
7,002-GABINETE DO PREF,2022-10-01,3718921,FRANCISCO JOSE GOUVEIA SOARESFRANCISCO ROGERIO...,GABINETE DO PREFEITOGABINETE DO PREFEITOGABINE...,ECONOMISTAAGENTE DE SUPORTE GERENCIALAGENTE DE...,395687.76
8,002-GABINETE DO PREF,2022-11-01,3787330,FRANCISCO JOSE GOUVEIA SOARESFRANCISCO ROGERIO...,GABINETE DO PREFEITOGABINETE DO PREFEITOGABINE...,ECONOMISTAAGENTE DE SUPORTE GERENCIALAUXILIAR ...,342151.64
9,002-GABINETE DO PREF,2022-12-01,3470663,JANETE CLARICE AMARAL DA COSTA ARNDTHELDER DED...,GABINETE DO PREFEITOGABINETE DO PREFEITOGABINE...,ASSESSOR ESPECIAL IIASSESSOR ESPECIAL IIIASSES...,315684.35


In [130]:
df_gastos_setor.sort_values( )

KeyError: ('Setor', 'Provento')

In [110]:
#importando dash plotly
from dash import Dash, html, dash_table, dcc, Input, Output, callback
import plotly.express as px
import matplotlib.pyplot as plt
import dash_bootstrap_components as dbc
from dash.dash_table.Format import Format, Scheme, Sign

In [127]:
STYLE01 = {'color':'white','text-align': 'center'}

#iniciar o app
external_stylesheets = [dbc.themes.CERULEAN]
app = Dash(__name__,external_stylesheets=external_stylesheets)

#layout
app.layout = dbc.Container([
    dbc.Row([
        html.Br([])
        ]),
    dbc.Row([
        dbc.Col([
            html.H1('Acompanhamento de Funcionarios da Caucaia-CE',style=STYLE01)
        ],width={"size": 8, "offset": 2}),
        dbc.Col([
            html.Img(src='https://www.caucaia.ce.gov.br/imagens/logo.png', style={'width': '150px'}),
        ],width={"size": 1, "order": "last"}),

        html.Br([]),
        html.Br([])

        ]),
    dbc.Row([
        dbc.Col([html.H4('Maior Salário Pago:', 
                         style=STYLE01),
                html.H6(f'R$ {provento_max_global}', 
                         style=STYLE01)
                         ]),
        dbc.Col([html.H4('Salário Médio:', 
                         style=STYLE01),
                html.H6(f'R$ {provento_mean_global:.2f}', 
                         style=STYLE01)
                         ]),
        dbc.Col([html.H4('Maior Gasto Mensal:', 
                         style=STYLE01),
                html.H6(f'R$ {provento_max_pago_mes_valor} ({provento_max_pago_mes_numero}/{provento_max_pago_ano_numero})', 
                         style=STYLE01)
                         ]),
        dbc.Col([html.H4('Setor com Maior Gasto:', 
                         style=STYLE01),
                html.H6(f'{setor_maior_gasto}', style=STYLE01)
                         ]),
        ]),
    
    dbc.Row([
            html.Br([])
             ]),
    
    dbc.Row([html.H4('Setores com Maiores Gastos:', 
                         style=STYLE01),
            dcc.Graph(figure=px.funnel(data_frame=df_gastos_setor.head(50), 
                                       x='Provento', y='Setor',
                                    #    range_x=[10000,92487290], 
                                       color = 'Date', 
                                    #    color_discrete_sequence = ['#c7254e'], 
                                       opacity = 1))
             ]),

    dbc.Row([html.H4('Setores com Maiores Gastos:', 
                         style=STYLE01),
            dcc.Graph(figure=px.funnel(data_frame=df_gastos_setor, 
                                       x='Provento', y='Setor', animation_frame='Date',
                                       range_x=[5000,100000], 
                                       color_discrete_sequence = ['#c7254e'], 
                                       opacity = 1))
             ]),


    html.Br([]),
    
    


   

], style={'backgroundColor':'#c7254e'})

#run
if __name__=='__main__':
    app.run(debug=True)